# Задача 1

Реализовать функции для расчета цены Европейкого или Американкого опциона используя биномиальную модель.

In [1]:
def gen_lattice(S0, u, d, N):
    """generates a binomial lattice
    
    for a given up, down, start value and number of steps (N).
    Resulting lattice has N+1 levels.
    """
    S = [float(S0)]

    for i in range(1, N+1):
        for j in range(0, i+1):
            S.append( S0 * d**j * u**(i-j) )

    return S

#def lattice_levels(S):
#    return int( round( (m.sqrt(8*len(S)+1)-1)/2 ) )

In [4]:
import sys
print (sys.version)
#if sys.version_info >= (3,4):
#    print( "with enums" )

from enum import Enum
class CallPut(Enum):
    call = 1
    put = 2

class ExerciseStyle(Enum):
    euro = 1
    amer = 2

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]


In [4]:
def pv_crr(amerEuro, callPut, S0, K, T, r, sigma, N):
    import math as m
    dt = T / N
    df = m.exp(-r * dt)

    u = m.exp(sigma * m.sqrt(dt))
    d = 1 / u
    p = ( m.exp(r * dt) - d ) / (u - d)

    S = gen_lattice(S0=S0, N=N, u=u, d=d)
    L = N+1 #lattice_levels(S)
    
    payoff = lambda x: max( 0, x - K ) if CallPut.call == callPut else max( 0, K - x)

    # Calculate payoff at the last lattice level
    for i in range(len(S)-L,len(S)):
        S[i]=payoff(S[i])

    # Go backwards, calculate extected value for prev node, based on known nodes
    # Calculate payoff at node k, based on expected value of S
    #пробегаемся с обратную сторону по узлам
    k=len(S)-L-1
    for i in reversed(range(1, L)):
        for j in range(0, i):
            if amerEuro ==  ExerciseStyle.euro :
                S[k] = df*(p*S[k+i] + (1-p)*S[k+i+1])    
            else:
                S[k] = max( df*(p*S[k+i] + (1-p)*S[k+i+1]), payoff(S[k]) )   
            k = k - 1
    
    return S[0], S

# parameters
S0 = 100.
T = 1.
r = 0.05
sigma = 0.20
K = 100.
N = 1000

es = ExerciseStyle.euro
pvC, _ = pv_crr(es, CallPut.call, S0=S0, K=K, T=T, r=r, sigma=sigma, N=N)
print( pvC )
pvP, _ = pv_crr(es, CallPut.put, S0=S0, K=K, T=T, r=r, sigma=sigma, N=N)
print( pvP )

es = ExerciseStyle.amer
pvC, _ = pv_crr(es, CallPut.call, S0=S0, K=K, T=T, r=r, sigma=sigma, N=N)
print( pvC )
pvP, _ = pv_crr(es, CallPut.put, S0=S0, K=K, T=T, r=r, sigma=sigma, N=N)
print( pvP )

10.448584103764572


5.571526553833685


10.448584103764572


6.089595282977953


# Задача 2

Напишите код для расчета цены европейского и барьерного опционов для модели Хестона

Сделайте расчет

    * implied волотильности для европейского опциона
    * цены барьерного опциона knock-out 
    

In [1]:
# Параметры оционов
S0 = 80.; K = 85.; T = 1.0
B=110
#
r = 0.05
# Параметры модели
sigma0 = 0.2  # значение волатальности в начальный момент времени
long_term_variance = 0.25  # долгосрочный средний уровень дисперсии
mean_revertion_rate = 0.02 # скорость возврата к среднему
vol_of_vol=0.05  # волатильность волатильности
corr=0.07 # корреляция случайных факторов в модели

Модель задается уравнениями:

$$ dS_t = \mu S_t\,dt + \sqrt{\nu_t} S_t\,dW_1 \,$$

$$ d\nu_t = \theta(\omega - \nu_t)dt + \xi \sqrt{\nu_t}\,dW_2$$

где:

   * $\omega$ - долгосрочный средний уровень (long-term mean) 
   * $\theta$ - скорость возврата к среднему 
   * $\xi$ - волатильность волатильности
   * $W_1 \,$ и $W_2 \,$ винеровские процессы, с корреляцией $\rho \,$.

$$ \left< dW_1\, dW_2 \right> = \rho dt $$

Есть разные схемы дискретизации для процесса $\nu$ в модели Хестона.

Для процесса $\nu$ в модели Хестона будем использовать такой вариант схемы дискретизации:

$$ \nu_{t+dt} = \left( \sqrt{\nu_t} + \frac{1}{2}\xi\sqrt{dt}z_2 \right)^2 + \theta (\omega - \nu_t) dt - \frac{1}{4}\xi^2 dt $$

где $z_2$ случайная величена распределенная по нормальному закону (соответствует процессу $W_2$ )

См. Rouah F D. Euler and Milstein discretization.  http://www.frouah.com/finance%20notes/Euler%20and%20Milstein%20Discretization.pdf
 раздел "2 Milstein Scheme"


Обратите внимание, что для генерации двух процессов нужно сгенерировать две случайные величены $z_1$ и $z_2$, которые бы были скорелированны с коэффицентом $\rho$

In [7]:
#Напишем функцию для вычисления стоимости опциона

import numpy as np
import math as m
import scipy.optimize
def standard_normal(I):
    z = np.random.standard_normal(I)
    mean = np.mean(z)
    std = np.std(z)
    return (z - mean)/std 

M = 360 # number of steps in time
I = 50000 # number of paths to generate
q=0
def mc_KO(S0, K, T, r, sigma, M, I, B,call_put): 
    dt = float(T) / M
    G=I
    #вычисление цены, если барьер есть
    if B is not None:
        if call_put==CallPut.call:
            if(S0>B):
                return 0
        else:
            if(S0<B):
                return 0
    
        S = [S0]
        v=[sigma**2]
        for t in range(1, M + 1):
            #на каждом шаге оставляем лишь те траектории S (и соответсвующие им траектории v), которые не вышли за пределы B
            #генерируем 2 сл.в. с заданной корреляцией
            z1 = standard_normal(G)
            z2= z1*corr+np.sqrt(1-corr**2)*standard_normal(G)
            v_next=(np.sqrt(v[t-1])+0.5*vol_of_vol*m.sqrt(dt)*z2)**2 + mean_revertion_rate*(long_term_variance-v[t-1])*dt - 0.25*dt*(vol_of_vol)**2
                                                       
            #S_b = S[t - 1] * np.exp((r - 0.5 * v_next) * dt + np.sqrt(v_next) * m.sqrt(dt) * z1)
            S_b = S[t - 1] * np.exp((r - 0.5 * v[t-1]) * dt + np.sqrt(v[t-1]) * m.sqrt(dt) * z1)

            if  call_put == CallPut.call:
                G = len(S_b[S_b < B])
                S.append(S_b[S_b < B])
                v.append(v_next[S_b < B])
            else:
                G = len(S_b[S_b > B])
                S.append(S_b[S_b > B])
                v.append(v_next[S_b > B])
        
        if call_put==CallPut.call:
            C = np.sum(m.exp(-r * T) * np.maximum(S[-1] - K, 0)) / I
        else:
            C = np.sum(m.exp(-r * T) * np.maximum(-S[-1] + K, 0)) / I
    
    #вычисление цены обычного европейского
    else:
        v = np.zeros((M + 1, I))
        v[0] = sigma**2
        S = np.zeros((M + 1, I))
        S[0] = S0
        for t in range(1, M + 1):
            #генерируем 2 сл.в. с заданной корреляцией
            z1 = standard_normal(I)
            z2= z1*corr+np.sqrt(1-corr**2)*standard_normal(I)
            v[t]=(np.sqrt(v[t-1])+0.5*vol_of_vol*m.sqrt(dt)*z2)**2 + mean_revertion_rate*(long_term_variance-v[t-1])*dt - 0.25*dt*(vol_of_vol)**2
                                                        
            #S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * m.sqrt(dt) * z1)
            S[t] = S[t - 1] * np.exp((r - 0.5 * v[t-1]) * dt + np.sqrt(v[t-1]) * m.sqrt(dt) * z1)
            
        if call_put==CallPut.call:
            C = np.sum(m.exp(-r * T) * np.maximum(S[-1] - K, 0)) / I
        else:
            C = np.sum(m.exp(-r * T) * np.maximum(-S[-1] + K, 0)) / I
    
    return C 

In [8]:
C=mc_KO(S0, K, T, r, sigma0, M, I, None,CallPut.call)
print("Цена европейского колл опциона: %f" %C)

Цена европейского колл опциона: 6.165642


In [9]:
Bar=mc_KO(S0, K, T, r, sigma0, M, I, B,CallPut.call)
print("Цена барьерного колл опциона: %f" %Bar)

Цена барьерного колл опциона: 2.424434


In [10]:
from scipy import stats
def N(x):
    return stats.norm.cdf(x, 0.0, 1.0)
def bsm_d1(S, K, T, r, q, sigma):
    return (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * m.sqrt(T))

def bsm_d2(S, K, T, r, q, sigma):
    return (np.log(S / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * m.sqrt(T))

def bsm_pv(isCall, S, K, T, r, q, sigma):
    d1 = bsm_d1(S, K, T, r, q, sigma)
    d2 = bsm_d2(S, K, T, r, q, sigma)
    if isCall==CallPut.call:
        return S * N(d1) * m.exp(-q * T) - K * m.exp(-r * T) * N(d2)
    else:
        return K * N(-d2) * m.exp(-r * T)  - S * m.exp(-q * T) * N(-d1)

#Найдем имплайд волатилити для европейского колла, чью цену уже посчитали

def implied_vol(sigma):
        return C-bsm_pv(CallPut.call, S0, K, T, r, q, sigma)
    
sigma=scipy.optimize.brentq(implied_vol, -1, 10)
print("Implied volatility для европейского колл опциона: %lf" %sigma)

Implied volatility для европейского колл опциона: 0.205565
